# Read initial CSV

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import datetime as dt
from pytz import timezone
from tqdm import tqdm


In [2]:
repertory = './data/'

In [ ]:
file = 'ar41_for_ulb.csv'
df = pd.read_csv(repertory+file, sep=';')

## Add weather data to csv file

#### Get weather data from openweathermap

In [ ]:

from urllib.request import urlopen
import json
import csv
import time
import datetime


# Timestamp 1 january 2023 UTC  is	1672531201
# Each hour, + 3600
# Each day, + 86400
# End, 15 september, UTC is 1694818801
# Duration : 258 days or 22287600 seconds
# Careful, changement d'heure happening in Belgium but not for UTC

file = 'csvWeather.csv'
with open(repertory+file,'w',newline='') as weather :
    writer = csv.writer(weather)
    # Write the columns names on the csv
    fields = ['date','time','temp','wind','humidity']
    writer.writerow(fields)
    # For Bxl, Gand and Bastogne, write temperature, wind speed and humidity for each hour and each day between 01/01/23 and 15/09/23
    for day in range(258):
        for hour in range(24):
            urlbx = 'https://history.openweathermap.org/data/2.5/history/city?lat=50.5&lon=4.20&start={}&cnt=1&appid=0ecabdca455f4f022ca58f1a5929a9b7'.format(1672531201+(day*86400)+(hour*3600))
            data = json.load(urlopen(urlbx))
            windspeed = data['list'][0]['wind']['speed']
            temp = data['list'][0]['main']['temp']
            humidity =  data['list'][0]['main']['humidity']
            dt = datetime.datetime.fromtimestamp(data['list'][0]['dt'])
            dt = str(dt).split()
            writer.writerow([dt[0],dt[1],temp,windspeed,humidity])

            urlgand = 'https://history.openweathermap.org/data/2.5/history/city?lat=51.03&lon=3.43&start={}&cnt=1&appid=0ecabdca455f4f022ca58f1a5929a9b7'.format(1672531201+(day*86400)+(hour*3600))
            data = json.load(urlopen(urlgand))
            windspeed = data['list'][0]['wind']['speed']
            temp = data['list'][0]['main']['temp']
            humidity =  data['list'][0]['main']['humidity']
            writer.writerow([dt[0],dt[1],temp,windspeed,humidity])

            urlbast = 'https://history.openweathermap.org/data/2.5/history/city?lat=50.0&lon=5.43&start={}&cnt=1&appid=0ecabdca455f4f022ca58f1a5929a9b7'.format(1672531201+(day*86400)+(hour*3600))
            data = json.load(urlopen(urlbast))
            windspeed = data['list'][0]['wind']['speed']
            temp = data['list'][0]['main']['temp']
            humidity =  data['list'][0]['main']['humidity']
            writer.writerow([dt[0],dt[1],temp,windspeed,humidity])

#### Combine our dataset with weather data 

In [ ]:
# Find the closest point to define which weather report is the most relevant 

def closest(point, locs):
    close = 9e20
    for j in locs :
        dist = 0
        for i in range (2) :
            dist += (point[i]-j[i])**2
        if np.sqrt(dist) < close :
            close = np.sqrt(dist)
            best = j
    return best

In [ ]:
pathWeather = repertory+'csvWeather.csv'
pathTrain = repertory+'ar41_for_ulb.csv'
weather = pd.read_csv(pathWeather)
train = pd.read_csv(pathTrain,sep=';')

In [ ]:
# Creating a series for each dataframe that contains the closest point of the 3 studied locations
# We chose 3 different towns in Belgium based on their average temperatures. 
# One in the south, rather cold, one in the center and one in the north, rather warm.

locs = [(50.5, 4.2),(51.0,3.43),(50.0,5.43)]
weather['closest'] = list(zip(weather['lat'], weather['lon']))
train['coord'] = list(zip(train['lat'], train['lon']))
train['closest'] = train.apply(lambda row: closest(row['coord'],locs),axis=1)

In [ ]:
train['date'] = train['timestamps_UTC'].str.split().str[0]
train['time'] = train['timestamps_UTC'].str.split().str[1].str[:2]
weather['time'] = pd.to_datetime(weather['time'], format='%H:%M:%S')
weather['time'] = weather['time'].dt.hour.apply(lambda x: str(x).zfill(2))
print(train.shape,weather.shape)
final = pd.merge(train,weather,on=['time','date','closest'],how='inner')
print(final)

In [ ]:
writer = pd.ExcelWriter('final2.xlsx',engine='xlsxwriter') #TODO
final2 = final[final['mapped_veh_id'] < 105]
final2.to_excel(writer,sheet_name='sheet1')
writer.close()
final.to_csv(repertory+'trains_weather.csv',sep=";")

## Filter impossible values

In [ ]:
#Read csv with weather data
file = 'trains_weather.csv'
df = pd.read_csv(repertory+file, sep=';')
df.info()

In [ ]:
def sep_temp_air(df) :
    upper = 200
    lower = -15 # Determine if it is an overflow (and should be removed) or not
    high = (df["RS_E_InAirTemp_PC1"] > upper) | (df['RS_E_InAirTemp_PC2'] > upper)
    df.loc[high,'hta'] = True
    low = (df["RS_E_InAirTemp_PC1"] < lower) | (df['RS_E_InAirTemp_PC2'] < lower)
    df.loc[low,'lta'] = True
    return df

def sep_temp_water(df) :
    upper = 100
    lower = -5 # Determine if it is an overflow (and should be removed) or not
    high = (df["RS_E_WatTemp_PC1"] > upper) | (df['RS_E_WatTemp_PC2'] > upper)
    df.loc[high,'htw'] = True
    low = (df["RS_E_WatTemp_PC1"] <= lower) | (df['RS_E_WatTemp_PC2'] <= lower)
    df.loc[low,'ltw'] = True
    return df

def sep_temp_oil(df) :
    upper = 200
    lower = -100 # Determine if it is an overflow (and should be removed) or not
    high = (df["RS_T_OilTemp_PC1"] > upper) | (df['RS_T_OilTemp_PC2'] > upper)
    df.loc[high,'hto'] = True
    low = (df["RS_T_OilTemp_PC1"] < lower) | (df['RS_T_OilTemp_PC2'] < lower)
    df.loc[low,'lto'] = True
    return df

def sep_rpm(df) :
    upper = 3000
    lower = -1
    high = (df["RS_E_RPM_PC1"] > upper) | (df['RS_E_RPM_PC2'] > upper)
    df.loc[high,'hrpm'] = True
    low = (df["RS_E_RPM_PC1"] < lower) | (df['RS_E_RPM_PC2'] < lower)
    df.loc[low,'lrpm'] = True
    return df

def sep_press(df) :
    upper = 689
    lower = 1 
    high = (df["RS_E_OilPress_PC1"] > upper) | (df['RS_E_OilPress_PC2'] > upper)
    df.loc[high,'hpress'] = True
    low = (df["RS_E_OilPress_PC1"] < lower) | (df['RS_E_OilPress_PC2'] < lower)
    df.loc[low,'lpress'] = True
    return df


### Cleaning DF

df = df.drop(['lat_y','lon_y','closest','time'], axis=1)
df = df.dropna()
df['mapped_veh_id'] = df['mapped_veh_id'].astype(int)
df['temp'] = df['temp'].astype(float)
df['wind'] = df['wind'].astype(float)
df['humidity'] = df['humidity'].astype(float)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df = sep_temp_water(df)
df = sep_temp_air(df)
df = sep_temp_oil(df)
df = sep_rpm(df)
df = sep_press(df)

df = df.fillna(False)

temp = ((df['hto'] == True) | (df['lto'] == True) | (df['hta'] == True) | (df['lta'] == True) | (df['htw'] == True) | (df['ltw'] == True))
pressrpm = ((df['hpress'] == True) | (df['lpress'] == True) | (df['hrpm'] == True) | (df['lrpm'] == True) )
invalid = temp | pressrpm
ruled_out = df[invalid]
fitting = df[~invalid]

fitting.to_csv(repertory+'fitting.csv',sep=';',index=True)
ruled_out.to_csv(repertory+'ruled_out.csv',sep=';',index=True)

In [ ]:
df = fitting

# Visualize histograms of PC1 and PC2 sensor by sensor

In [ ]:
B = 2**6

column_names = ['mapped_veh_id', 'lat','lon','RS_E_InAirTemp_PC1', 'RS_E_OilPress_PC1', 'RS_E_RPM_PC1','RS_E_WatTemp_PC1','RS_T_OilTemp_PC1']

# 0 => mapped_veh_id
# 1 => lat
# 2 => lon
# 3 => RS_E_InAirTemp_PC1
# 4 => RS_E_OilPress_PC1
# 5 => RS_E_RPM_PC1
# 6 => RS_E_WatTemp_PC1
# 7 => RS_E_OilTemp_PC1

selected_column = str(column_names[7])[0:-4] #select one of the sensors
selected_columnPC1 = selected_column+'_PC1'
selected_columnPC2 = selected_column+'_PC2'

print("selected data column: "+selected_column) #announce selected data

data_col_pc1 = df[selected_columnPC1]
data_col_pc2 = df[selected_columnPC2]
data_col = pd.concat([data_col_pc1,data_col_pc2])
# Adjust the number of bins and other parameters as needed
plt.hist(data_col_pc1, bins=B, color='skyblue', alpha=0.4, edgecolor='black')
plt.hist(data_col_pc2, bins=B, color='orange', alpha=0.4, edgecolor='black')

# Customize the plot with labels and title
plt.xlabel(selected_column)
plt.ylabel('Frequency')
plt.title(selected_column + ' Histogram N='+str(df.shape[0]))

# Calculate mean, median, and mode for PC1
mean_value = data_col.mean()
median_value = data_col.median()
mode_value = data_col.mode().values[0]
std_value = data_col.std()

# Plot mean, median, and mode for PC1
plt.axvline(mean_value, color='red', linestyle='dashed', linewidth=1, label='Mean')
plt.axvline(median_value, color='green', linestyle='dashed', linewidth=1, label='Median')
plt.axvline(mode_value, color='blue', linestyle='dashed', linewidth=1, label='Mode')
plt.axvline(mean_value-2*std_value, color='purple', linestyle='dashed', linewidth=1)
plt.axvline(mean_value+2*std_value, color='purple', linestyle='dashed', linewidth=1, label='2std')

# Generate x values for the normal curve for PC1
x = np.linspace(data_col.min(), data_col.max(), 100)
# Calculate the corresponding y values for the normal curve using mean and std
y = stats.norm.pdf(x, mean_value, std_value)
# Scale the y values to match the histogram
y = y * len(data_col) *np.diff(plt.hist(data_col, bins=B, color='skyblue',alpha=0)[1][0:2])/2
# Plot the normal curve for data_col
plt.plot(x, y, color='red', label='Normal Distribution')
plt.legend()
# Display the histogram with the normal curves
plt.show()

# Exploring data with common correlations metrix : Pearson correlation coefficient

In [ ]:
df[['RS_E_InAirTemp_PC1','RS_E_WatTemp_PC1','RS_T_OilTemp_PC1','RS_E_OilPress_PC1','RS_E_RPM_PC1', 'temp', 'wind', 'humidity']].corr('pearson')

In [ ]:
df[['RS_E_InAirTemp_PC2','RS_E_WatTemp_PC2','RS_T_OilTemp_PC2','RS_E_OilPress_PC2','RS_E_RPM_PC2', 'temp', 'wind', 'humidity']].corr('pearson')

In [ ]:
df[['RS_E_InAirTemp_PC1','RS_E_WatTemp_PC1','RS_T_OilTemp_PC1','RS_E_OilPress_PC1','RS_E_RPM_PC1', RS_E_InAirTemp_PC2','RS_E_WatTemp_PC2','RS_T_OilTemp_PC2','RS_E_OilPress_PC2','RS_E_RPM_PC2', 'temp', 'wind', 'humidity']].corr('pearson')

# Principal component analysis

In [ ]:
from sklearn.decomposition import PCA

# You must normalize the data before applying the fit method
df_normalized=(df[['RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC1',
       'RS_E_OilPress_PC2', 'RS_E_RPM_PC1', 'RS_E_RPM_PC2', 'RS_E_WatTemp_PC1',
       'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2', 'temp', 'wind', 'humidity']] - df_sub[['RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC1',
       'RS_E_OilPress_PC2', 'RS_E_RPM_PC1', 'RS_E_RPM_PC2', 'RS_E_WatTemp_PC1',
       'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2', 'temp', 'wind', 'humidity']].mean()) / df_sub[['RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC1',
       'RS_E_OilPress_PC2', 'RS_E_RPM_PC1', 'RS_E_RPM_PC2', 'RS_E_WatTemp_PC1',
       'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2', 'temp', 'wind', 'humidity']].std()

pca = PCA(n_components=df[['RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC1',
       'RS_E_OilPress_PC2', 'RS_E_RPM_PC1', 'RS_E_RPM_PC2', 'RS_E_WatTemp_PC1',
       'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2', 'temp', 'wind', 'humidity']].shape[1])

df_pca_ = pca.fit(df_normalized)

# Reformat and view results
loadings = pd.DataFrame(pca.components_.T,
columns=['PC%s' % _ for _ in range(len(df_normalized.columns))],
index=df_normalized.columns)
print(loadings)

plt.plot(pca.explained_variance_ratio_)
plt.ylabel('Explained Variance')
plt.xlabel('Components')
plt.show()

# Local outlier factor

In [ ]:
# from sklearn.neighbors import LocalOutlierFactor
# 
# clf = LocalOutlierFactor(n_neighbors=5, contamination='auto', n_jobs=6)
# y_pred = clf.fit_predict(df_pca_)
# 
# X_scores = clf.negative_outlier_factor_
# TODO

## Create time series by train and with regular interval of time between two consecutive timestamps 
### (we consider a new time series when, between two timestamps, the train change or the difference of time is greater than 2 minutes and 30 seconds)

In [ ]:
'''Visualize occurence of time difference between chronological rows'''

df['timestamps_UTC'] = pd.to_datetime(df['timestamps_UTC'], utc=True)
tzone = timezone('Europe/Paris')
df['timestamps_UTC'] = df['timestamps_UTC'].dt.tz_convert(tzone)

#remove nan
print(df.shape)
df = df.dropna()
print(df.shape)

df = df.sort_values(by=['mapped_veh_id','timestamps_UTC'])
time_diffs = df['timestamps_UTC'].diff().dt.total_seconds()

custom_bins = np.concatenate([np.arange(0, 130, 1)])

# Plot a histogram of the time differences with custom bin edges
plt.hist(time_diffs, bins=custom_bins, edgecolor='black')
plt.yscale("log")
plt.xlabel('Time Differences (seconds)')
plt.ylabel('Frequency')
plt.title('Histogram of Time Differences')
plt.show()

In [1]:
'''Looking fo the best tau : average TS length vs tau-value'''
tau = 20-np.arange(1,20,1)
meanTSlength = list()
df = df.sort_values(by=['mapped_veh_id','timestamps_UTC'])
for i in range(len(tau)):
    df = df[((df['n_n1'] >= 60-tau[i]) & (df['n_n1'] <= 60+tau[i])) | (df['n_n1'] >= 120-tau[i]) & (df['n_n1'] <= 120+tau[i])]
    onetrain = df
    onetrain['timestamps_UTC'] = pd.to_datetime(onetrain['timestamps_UTC'], utc=True)
    onetrain['sequence_start'] = onetrain['timestamps_UTC'].diff().dt.total_seconds() > 150

    onetrain['traj'] = onetrain['sequence_start'].cumsum()
    #print(onetrain['traj'].drop_duplicates())
    onetrain = onetrain.drop(['sequence_start','date','time'],axis = 1)
    onetrain = onetrain[['mapped_veh_id','timestamps_UTC','traj']]
    meantrajlength = onetrain.groupby('traj').count().agg('mean') 
    meanTSlength.append(meantrajlength)
    onetraine=onetrain.drop(['traj'],axis=1)

# Plot the lists
plt.plot(tau,meanTSlength)

plt.title('Average Time Series Length')

# Add labels and title
plt.xlabel('tau')
plt.ylabel('mean(len(TS))')

# Display the plot
plt.show()

'Looking fo the best tau'

In [ ]:
''' Add n_n1 to df '''
#n_n1 represents the time difference T(n) between samples n and n_1 (first item is 0)
df = df.sort_values(by=['mapped_veh_id','timestamps_UTC']).reset_index()
df['timestamps_UTC'] = pd.to_datetime(df['timestamps_UTC'], utc=True)
#tzone = timezone('Europe/Paris')
#df['timestamps_UTC'] = df['timestamps_UTC'].dt.tz_convert(tzone)

n_n1 = df['timestamps_UTC'].diff().dt.total_seconds()
n_n1 = n_n1.to_frame(name='n_n1')
n_n1 = n_n1.fillna(0)
df.insert(3,'n_n1',n_n1)
#drop time intervals 
tau = 10
df = df[((df['n_n1'] >= 60-tau) & (df['n_n1'] <= 60+tau)) | (df['n_n1'] >= 120-tau) & (df['n_n1'] <= 120+tau)]

onetrain = df.sort_values(by=['mapped_veh_id','timestamps_UTC']).reset_index()
onetrain['timestamps_UTC'] = pd.to_datetime(onetrain['timestamps_UTC'], utc=True)

In [ ]:
'''Adding a trajets column '''

onetrain['sequence_start'] = onetrain['timestamps_UTC'].diff().dt.total_seconds() > 150
onetrain['traj'] = onetrain['sequence_start'].cumsum()
#print(onetrain['traj'].drop_duplicates())
onetrain = onetrain.drop(['sequence_start','date','time'],axis = 1)
#print(onetrain[['traj','timestamps_UTC','n_n1']].head(10))
df2 = pd.DataFrame(columns=onetrain.columns)

In [ ]:
'''Interpolate missing rows'''

global add
add = -1

def gb_to_rows(gb):
    gb.apply(lambda w: addrow(w,df2),axis=1)
    print(gb['traj'].drop_duplicates())
def addrow(row,df2):
    global add
    if row['n_n1'] > 90:
        df2.loc[add,'timestamps_UTC'] = row['timestamps_UTC'] + dt.timedelta(seconds=60)
        df2.loc[add,'mapped_veh_id'] = row['mapped_veh_id']
        df2.loc[add,'traj'] = row['traj']
        add -= 1
import datetime as dt
onetrain.groupby('traj').apply(lambda z: gb_to_rows(z))
df = pd.concat([onetrain,df2])
df=df.interpolate()

# Optimization of filter to have bigger timeseries with regular interval times between rows

In [4]:
# # Sort the DataFrame by train_id and timestamps in chronological order
# df = df.sort_values(by=['mapped_veh_id','timestamps_UTC']).reset_index()

# # Convert timestamps to datetime objects
# df['timestamps_UTC'] = pd.to_datetime(df['timestamps_UTC'], utc=True)

# # Calculate time differences (T(n)) between consecutive samples and fill NaN with 0
# n_n1 = df['timestamps_UTC'].diff().dt.total_seconds()
# n_n1 = n_n1.to_frame(name='n_n1')
# n_n1 = n_n1.fillna(0)

# # Insert the calculated time differences as a new column 'n_n1' at position 3
# df.insert(3,'n_n1',n_n1)

# df2 = df.copy()


In [5]:
# import pytz
# from datetime import timezone

# now = pd.datetime.now(tz=pytz.UTC)


# # Convert the timestamps_UTC column to datetime objects
# df2['timestamps_UTC'] = pd.to_datetime(df2['timestamps_UTC'], errors='coerce')

# # Create a new column 'bin_interval_time' containing the seconds part
# df2['bin_interval_time'] = df2['timestamps_UTC'].dt.second

# print(df2.head(100))

# # Sort the DataFrame by train_id and timestamps again
# sortedByTrain2 = df2.sort_values(by=['mapped_veh_id','timestamps_UTC']).reset_index()

# # Convert timestamps to datetime objects
# sortedByTrain2['timestamps_UTC'] = pd.to_datetime(sortedByTrain2['timestamps_UTC'], utc=True)

# # Add a 'sequence_start' column based on time gaps greater than 150 seconds
# sortedByTrain2['sequence_start'] = sortedByTrain2['n_n1'] > 150 

# sortedByTrain2['traj'] = sortedByTrain2['sequence_start'].cumsum()

# sortedByTrain2 = sortedByTrain2.drop(['sequence_start','date','time'],axis = 1)

# print(sortedByTrain2[['traj','timestamps_UTC','n_n1']].head(100))

# sortedByTrain2['most_common_bin'] = sortedByTrain2.groupby('traj')['bin_interval_time'].transform(lambda x: x.mode().iloc[0])

# print(df2.head(100))

C:\Users\Vince\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.


       index  Unnamed: 0.1  mapped_veh_id  n_n1            timestamps_UTC  \
0   10519630       5493376          102.0   0.0 2023-01-23 07:25:08+00:00   
1   10519642       8969009          102.0   8.0 2023-01-23 07:25:16+00:00   
2   10519662      13873566          102.0  21.0 2023-01-23 07:25:37+00:00   
3   10519663      14994675          102.0   4.0 2023-01-23 07:25:41+00:00   
4   10519656      11935795          102.0  29.0 2023-01-23 07:26:10+00:00   
..       ...           ...            ...   ...                       ...   
95  10539750       3691099          102.0   6.0 2023-01-23 14:52:14+00:00   
96  10540524       8960631          102.0  54.0 2023-01-23 14:53:08+00:00   
97  10539849       4395399          102.0   6.0 2023-01-23 14:53:14+00:00   
98  10539500       2406388          102.0  55.0 2023-01-23 14:54:09+00:00   
99  10541729      16102555          102.0   6.0 2023-01-23 14:54:15+00:00   

        lat_x     lon_x  RS_E_InAirTemp_PC1  RS_E_InAirTemp_PC2  \
0   51.0

In [17]:
# sortedByTrain2 = sortedByTrain
# sortedByTrain2["diff_bin"] = abs(sortedByTrain2["bin_interval_time"] - sortedByTrain2["most_common_bin"])

# tau = 10

# sortedByTrain2 = sortedByTrain2[sortedByTrain2["diff_bin"] <= tau]

# sortedByTrain2.drop("n_n1", inplace=True, axis = 1)

# # Calculate time differences (T(n)) between consecutive samples and fill NaN with 0
# n_n1 = sortedByTrain2['timestamps_UTC'].diff().dt.total_seconds()
# n_n1 = n_n1.to_frame(name='n_n1')
# n_n1 = n_n1.fillna(0)

# # Insert the calculated time differences as a new column 'n_n1' at position 3
# sortedByTrain2.insert(3,'n_n1',n_n1)

# sortedByTrain2 = sortedByTrain2[sortedByTrain2["n_n1"] >= 2 * tau]

c:\Users\Vince\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [27]:
# df_final = pd.DataFrame(columns=sortedByTrain2.columns)
# print(df_final)
# global add 
# add = -1 

# def gb_to_rows(gb) :
#     gb.apply(lambda w: addrow(w, df_final), axis=1)
#     progress_bar1.update(1)
    
# def addrow(row, df_final) :
#     global add
#     for i in range(round(row['n_n1'] / 60) - 1): # to check
#         df_final.loc[add + i,'timestamps_UTC'] = row['timestamps_UTC'] + dt.timedelta(seconds=i * 60)
#         df_final.loc[add + i,'mapped_veh_id'] = row['mapped_veh_id']
#         df_final.loc[add + i,'traj'] = row['traj']
#         add -= 1

# progress_bar1.close()
# from tqdm import tqdm
# progress_bar1 = tqdm(total=len(sortedByTrain2['traj'].unique()), desc="Processing groups")
# import datetime as dt
# sortedByTrain2.groupby('traj').apply(lambda z: gb_to_rows(z))
# progress_bar1.close()


Empty DataFrame
Columns: [level_0, index, Unnamed: 0.1, n_n1, mapped_veh_id, timestamps_UTC, lat_x, lon_x, RS_E_InAirTemp_PC1, RS_E_InAirTemp_PC2, RS_E_OilPress_PC1, RS_E_OilPress_PC2, RS_E_RPM_PC1, RS_E_RPM_PC2, RS_E_WatTemp_PC1, RS_E_WatTemp_PC2, RS_T_OilTemp_PC1, RS_T_OilTemp_PC2, coord, closest, temp, wind, humidity, lat_y, lon_y, bin_interval_time, traj, most_common_bin, diff_bin]
Index: []

[0 rows x 29 columns]


In [22]:
# sortedByTrain3 = sortedByTrain2
# import datetime as dt
# sortedByTrain2.groupby('traj').apply(lambda z: gb_to_rows(z))

4    0
Name: traj, dtype: int32
8    1
Name: traj, dtype: int32
980    2
Name: traj, dtype: int32
991    3
Name: traj, dtype: int32
1892    4
Name: traj, dtype: int32
2026    5
Name: traj, dtype: int32
2032    6
Name: traj, dtype: int32
2052    7
Name: traj, dtype: int32
3670    8
Name: traj, dtype: int32
3672    9
Name: traj, dtype: int32
3681    10
Name: traj, dtype: int32
4107    11
Name: traj, dtype: int32
5613    12
Name: traj, dtype: int32
6511    13
Name: traj, dtype: int32
6531    14
Name: traj, dtype: int32
6582    15
Name: traj, dtype: int32
6590    16
Name: traj, dtype: int32
6609    17
Name: traj, dtype: int32
6639    18
Name: traj, dtype: int32
7065    19
Name: traj, dtype: int32
7942    20
Name: traj, dtype: int32
9283    21
Name: traj, dtype: int32
9816    22
Name: traj, dtype: int32
10175    23
Name: traj, dtype: int32
11045    24
Name: traj, dtype: int32
11049    25
Name: traj, dtype: int32
11560    26
Name: traj, dtype: int32
11756    27
Name: traj, dtype: int32
12115

KeyboardInterrupt: 

# Time series analysis : Auto correlation function and partial auto correlation function
### (Allows to find p and q for ARIMA : p = autoregression order and q = moving average order)

In [ ]:
traj1 = df[df['traj']==7809]
traj2 = df[df['traj']==89117]
tsmatrix0 = traj1[['RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC1','RS_E_OilPress_PC2', 'RS_E_RPM_PC1', 'RS_E_RPM_PC2', 'RS_E_WatTemp_PC1','RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2']]
d = 0  #Differencing order SARIMA(p,1,q)
if d == 0:
    tsmatrix = tsmatrix0
else:
    tsmatrix = tsmatrix0.diff(periods=d).dropna()

time_range = np.arange(0, tsmatrix.shape[0]/60, 1/60) #time in hours

'''time series plotting'''
for i in range(0, 10, 2):
    plt.figure(figsize=(8, 4))
    plt.plot(time_range, tsmatrix.iloc[:, i], color='blue', label='TS {}'.format(tsmatrix.columns[i]))
    plt.plot(time_range, tsmatrix.iloc[:, i+1], color='orange', label='TS {}'.format(tsmatrix.columns[i+1]))
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.title('Columns {} and {}'.format(tsmatrix.columns[i], tsmatrix.columns[i+1]))
    plt.legend()
    plt.show()
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
for i in range(0,10,2):
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(tsmatrix.iloc[:, i], lags=100, ax=ax[0], title='ACF for TS {}'.format(tsmatrix.columns[i+1]))
    plot_pacf(tsmatrix.iloc[:, i], lags=100, ax=ax[1], title='PACF for TS {}'.format(tsmatrix.columns[i+1]))
    plt.show()

# Check stationarity or non stationarity of time series
### (Using ADF and KPSS test)

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm

def test_stationarity_adf(ts_data, column='', signif=0.05, series=False):
    if series:
        adf_test = adfuller(ts_data, autolag='AIC')
    else:
        adf_test = adfuller(ts_data[column], autolag='AIC')
    p_value = adf_test[1]
    if p_value <= signif:
        test_result = "Stationary"
    else:
        test_result = "Non-Stationary"
    return test_result

In [ ]:
def test_stationarity_kpss(ts_data, column='', signif=0.05, series=False):
    if series:
        kpss_test = kpss(ts_data)
    else:
        kpss_test = kpss(ts_data[column])
    p_value = kpss_test[1]
    if p_value <= signif:
        test_result = "Non-Stationary"
    else:
        test_result = "Stationary"
    return test_result

In [ ]:
print(test_stationarity_adf(traj1, 'RS_E_InAirTemp_PC1'))
print(test_stationarity_kpss(traj1, 'RS_E_InAirTemp_PC1'))

In [ ]:
def differencing(data, column, order):
    differenced_data = data[column].diff(order)
    differenced_data.fillna(differenced_data.mean(), inplace=True)
    return differenced_data

preprocessed_data = differencing(traj1, 'RS_E_InAirTemp_PC1', 1)
print(preprocessed_data)

In [ ]:
preprocessed_data = differencing(traj1, 'RS_E_InAirTemp_PC1', 2)
test_stationarity_adf(preprocessed_data, series=True)
test_stationarity_kpss(preprocessed_data, series=True)

# Implementation of the ARIMA model with pmdarima

In [ ]:
import pmdarima as pm


model = pm.auto_arima(traj1['RS_E_InAirTemp_PC1'],
                      max_p=12, q_start=0 ,test='kpss', p_start=3,
                      suppress_warnings=True, 
                       stepwise=True, trace=True, information_criterion='aic')
training=traj1['RS_E_InAirTemp_PC1']
model_fit = model.fit(training)

In [ ]:
a = model_fit.params()
AR = a[0:4]
print(AR)

In [ ]:
testing=traj2['RS_E_InAirTemp_PC1']
dtest = testing-testing.shift(1).fillna(0)
y = AR[0]*dtest.shift(0)

for p in range(len(AR)):
    if p>0:
        y = y+AR[p]*dtest.shift(-p)

y = y.cumsum()
y = y - np.mean(y-testing)
plt.figure(figsize=(10, 6))
plt.plot(time_range,y, label='model')
plt.plot(time_range,testing, label='testing')
plt.plot(time_range,y-testing, label='error')

plt.xlabel('Time in Hrs')
plt.ylabel('Value')
plt.title('Comparison of y and traj2["air"]')
plt.legend()
plt.show()
sum(y.fillna(0)-testing)

# Continuity testing (naïve method) using isclose() and shift()

In [ ]:
anomalies = pd.DataFrame(columns=df.columns)
dataprop = list()
colum = 'RS_E_InAirTemp_PC1'
rt=0.59

N = anomalies.shape[0]
anomalies = pd.merge(anomalies,df[(df['traj'].shift(1)==df['traj']) & (~np.isclose(df[colum].shift(1),df[colum],rtol=rt))],how='outer')
print(colum,anomalies.shape[0]-N)

print(rt,anomalies.shape[0]/df.shape[0])

traj_id = anomalies['traj'].value_counts().index[0]
traj = df[df['traj']==traj_id]
traj = traj[['timestamps_UTC',colum]]
anom = anomalies[anomalies['traj']==traj_id]
anom = anom[['timestamps_UTC',colum]]

plt.figure(figsize=(10, 6))
plt.plot(traj['timestamps_UTC'], traj[colum], label='Traj Data', color='b')
plt.scatter(anom['timestamps_UTC'], anom[colum], label='Anomalies', color='r')
plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.title('Timeseries with anomalies in '+colum)
plt.legend()
plt.show()

# Detection of outlier using Prophet